### QA. SQUAD dataset

You can work with Squad datasets (any other available in hugging face) in different formats. 
* via library datasets
* via already prepared pipelines
* directly with data loaders to classify and finetune. 



In [1]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 194kB 16.9MB/s 
     |████████████████████████████████| 245kB 41.3MB/s 
     |████████████████████████████████| 112kB 53.7MB/s 
     |████████████████████████████████| 2.0MB 17.1MB/s 
     |████████████████████████████████| 890kB 45.8MB/s 
     |████████████████████████████████| 3.2MB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f4e50f7c325360fbaa3fdc63229d4ece2241523a04efe1f40bc6eef7426af8e1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric
from pprint import pprint

datasets = list_datasets()
metrics = list_metrics()

# Print all the available datasets
pprint(datasets, compact=True)
# List all the available metrics
pprint(metrics, compact=True)

['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc',
 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue',
 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity',
 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'amttl', 'anli',
 'app_reviews', 'aqua_rat', 'aquamuse', 'ar_cov19', 'ar_res_reviews',
 'ar_sarcasm', 'arabic_billion_words', 'arabic_pos_dialect',
 'arabic_speech_corpus', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset',
 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'atomic', 'autshumato',
 'babi_qa', 'bbc_hindi_nli', 'bc2gm_corpus', 'best2009', 'bianet', 'bible_para',
 'big_patent', 'billsum', 'bing_coronavirus_query_set', 'biomrc',
 'blended_skill_talk', 'blimp', 'blog_authorship_corpus', 'bn_hate_speech',
 'bookcorpus', 'bookcorpusopen', 'boolq', 'bprec', 'break_data', 'brwac',
 'bsd_ja_en', 'bswac', 'c3', 'c4', 'cail2018', 'caner', 'capes',
 'catalonia_independence', 'cawac', 'cbt', 'cc100', 'cc_news',
 'ccaligned_

In [3]:
# Load a dataset and print the first example in the training set
squad_dataset = load_dataset('squad')
print(squad_dataset['train'][0])

# Load a metric
squad_metric = load_metric('squad')

print(squad_dataset["train"])
# Process the dataset - add a column with the length of the context texts
dataset_with_length = squad_dataset.map(lambda x: {"length": len(x["context"])})

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9. Subsequent calls will reuse this data.
{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'id': '57


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})


In [4]:
# Process the dataset - tokenize the context texts
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenized_dataset = squad_dataset.map(lambda x: tokenizer(x['context']), batched=True)
tokenized_dataset["train"]

Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['answers', 'attention_mask', 'context', 'id', 'input_ids', 'question', 'title', 'token_type_ids'],
    num_rows: 87599
})

In [5]:
squad_dataset = list_datasets(with_details=True)[datasets.index('squad')]

pprint(squad_dataset.__dict__)

{'author': None,
 'citation': '@article{2016arXiv160605250R,\n'
             '       author = {{Rajpurkar}, Pranav and {Zhang}, Jian and '
             '{Lopyrev},\n'
             '                 Konstantin and {Liang}, Percy},\n'
             '        title = "{SQuAD: 100,000+ Questions for Machine '
             'Comprehension of Text}",\n'
             '      journal = {arXiv e-prints},\n'
             '         year = 2016,\n'
             '          eid = {arXiv:1606.05250},\n'
             '        pages = {arXiv:1606.05250},\n'
             'archivePrefix = {arXiv},\n'
             '       eprint = {1606.05250},\n'
             '}',
 'description': 'Stanford Question Answering Dataset (SQuAD) is a reading '
                'comprehension dataset, consisting of questions posed by '
                'crowdworkers on a set of Wikipedia articles, where the answer '
                'to every question is a segment of text, or span, from the '
                'corresponding reading pa

In [ ]:
import torch 
from datasets import load_dataset

dataset = load_dataset('squad')

def get_correct_alignement(context, answer):
    """ Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here. """
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = start_idx + len(gold_text)
    if context[start_idx:end_idx] == gold_text:
        return start_idx, end_idx       # When the gold label position is good
    elif context[start_idx-1:end_idx-1] == gold_text:
        return start_idx-1, end_idx-1   # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
        return start_idx-2, end_idx-2   # When the gold label is off by two character
    else:
        raise ValueError()

# Tokenize our training dataset
def convert_to_features(example_batch):
    # Tokenize contexts and questions (as pairs of inputs)
    encodings = tokenizer(example_batch['context'], example_batch['question'], truncation=True)

    # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methods.
    start_positions, end_positions = [], []
    for i, (context, answer) in enumerate(zip(example_batch['context'], example_batch['answers'])):
        start_idx, end_idx = get_correct_alignement(context, answer)
        start_positions.append(encodings.char_to_token(i, start_idx))
        end_positions.append(encodings.char_to_token(i, end_idx-1))
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

encoded_dataset = dataset.map(convert_to_features, batched=True)

# Format our dataset to outputs torch.Tensor to train a pytorch model
columns = ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']
encoded_dataset.set_format(type='torch', columns=columns)

# Instantiate a PyTorch Dataloader around our dataset
# Let's do dynamic batching (pad on the fly with our own collate_fn)
def collate_fn(examples):
    return tokenizer.pad(examples, return_tensors='pt')
dataloader = torch.utils.data.DataLoader(encoded_dataset['train'], collate_fn=collate_fn, batch_size=8)

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9)


In [7]:
from transformers import BertForQuestionAnswering
import torch

# Bert Model with a span classification head on top for extractive question-answering tasks 
# a linear layers on top of the hidden-states output to compute span start logits and span end logits
model = BertForQuestionAnswering.from_pretrained('bert-base-cased', return_dict=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
# Now let's train our model
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.train().to(device)
for i, batch in enumerate(dataloader):
    batch.to(device)
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    model.zero_grad()
    print(f'Step {i} - loss: {loss:.3}')
    if i > 3:
        break

#### pipelines

In [9]:
from transformers import pipeline

nlp = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""

In [10]:
result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

result = nlp(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'the task of extracting an answer from a text given a question', score: 0.6226, start: 34, end: 95
Answer: 'SQuAD dataset', score: 0.5053, start: 147, end: 160


Here is an example of question answering using a model and a tokenizer.

In [11]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it with the weights stored in the checkpoint.
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")


# Define a text and a few questions.
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
     "How many pretrained models are available in 🤗 Transformers?",
     "What does 🤗 Transformers provide?",
     "🤗 Transformers provides interoperability between which frameworks?",
]

# Iterate over the questions and build a sequence from the text and the current question, 
# with the correct model-specific separators token type ids and attention masks.
for question in questions:
     inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
     input_ids = inputs["input_ids"].tolist()[0]

     # Pass this sequence through the model. This outputs a range of scores across the entire
     # sequence tokens (question and text), for both the start and end positions.
     outputs = model(**inputs)
     answer_start_scores = outputs.start_logits
     answer_end_scores = outputs.end_logits

     # Compute the softmax of the result to get probabilities over the tokens.
     answer_start = torch.argmax(
         answer_start_scores
     )  # Get the most likely beginning of answer with the argmax of the score
     answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
     answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

     print(f"Question: {question}")
     print(f"Answer: {answer}")


Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch


### Russian QA models

You can you DeepPavlov models for Russian QA.
* For SQAUD dataset http://docs.deeppavlov.ai/en/master/features/models/squad.html
* For KBQA http://docs.deeppavlov.ai/en/master/features/models/kbqa.html
* Fir ODQA http://docs.deeppavlov.ai/en/master/features/skills/odqa.html


In [12]:
!pip install deeppavlov

     |████████████████████████████████| 993kB 20.1MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 6.7MB 52.3MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 10.4MB 51.5MB/s 
     |████████████████████████████████| 655kB 49.1MB/s 
     |████████████████████████████████| 1.5MB 46.5MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 512kB 51.6MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 8.2MB 34.5MB/s 
     |████████████████████████████████| 860kB 55.2MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 2.1MB 45.5MB/s 
     |████████████████████████████████| 7.3MB 32.3MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     

In [ ]:
# Demo is here https://demo.deeppavlov.ai/#/ru/textqa

!python -m deeppavlov install squad_ru_rubert_infer
# !python -m deeppavlov download squad_ru_rubert_infer

In [ ]:
!pip install --upgrade tensorflow-gpu==2.2.0 --user
!pip uninstall tensorflow-datasets
!pip install tensorflow-datasets==4.0.0
# reconnect colab

In [ ]:
from deeppavlov import build_model, configs

model = build_model(configs.squad.squad_ru_rubert_infer, download=True)
model(['DeepPavlov это библиотека для NLP задач и постороения диалоговых систем.'], ['Что такое DeepPavlov?'])

2021-03-25 19:22:59.170 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_rubert.tar.gz download because of matching hashes
2021-03-25 19:23:01.256 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!












The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-03-25 19:23:35.884 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_ru_bert/model_rubert]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_ru_bert/model_rubert


[['библиотека для NLP задач и постороения диалоговых систем'],
 [15],
 [1729423.0]]

In [13]:
!git clone https://github.com/deepmipt/DeepPavlov.git

Cloning into 'DeepPavlov'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 57169 (delta 74), reused 64 (delta 28), pack-reused 57038
Receiving objects: 100% (57169/57169), 37.15 MiB | 33.48 MiB/s, done.
Resolving deltas: 100% (43902/43902), done.


In [14]:
!cat ./DeepPavlov/deeppavlov/configs/squad/squad_ru_rubert_infer.json

{
  "dataset_reader": {
    "class_name": "squad_dataset_reader",
    "dataset": "SberSQuADClean",
    "url": "http://files.deeppavlov.ai/datasets/sber_squad_clean-v1.1.tar.gz",
    "data_path": "{DOWNLOADS_PATH}/squad_ru_clean/"
  },
  "dataset_iterator": {
    "class_name": "squad_iterator",
    "seed": 1337,
    "shuffle": true
  },
  "chainer": {
    "in": ["context_raw", "question_raw"],
    "in_y": ["ans_raw", "ans_raw_start"],
    "pipe": [
      {
        "class_name": "squad_bert_infer",
        "lang": "ru",
        "batch_size": 128,
        "squad_model_config": "{CONFIGS_PATH}/squad/squad_ru_rubert.json",
        "vocab_file": "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
        "do_lower_case": false,
        "max_seq_length": 256,
        "in": ["context_raw", "question_raw"],
        "out": ["ans_predicted", "ans_start_predicted", "logits"]
      }
    ],
    "out": ["ans_predicted", "ans_start_predicted", "logits"]
  },
  "train": {
    "sh

In [ ]:
# Training. Check config!
from deeppavlov import train_model 

model = train_model("./DeepPavlov/deeppavlov/configs/squad/squad_ru_rubert_infer.json", download=True)